In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [20]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [21]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 512),
            nn.LayerNorm(512),
            nn.LeakyReLU(),
            nn.Dropout(0.1),

            nn.Linear(512, 512),
            nn.LayerNorm(512),
            nn.LeakyReLU(),
            nn.Dropout(0.1),


            nn.Linear(512, 10),
            nn.Softmax(1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=512, out_features=10, bias=True)
    (9): Softmax(dim=1)
  )
)


In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.00005)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [25]:
epochs = 7
with torch.no_grad():
    nn.init.xavier_uniform_(model.linear_relu_stack[0].weight)
    nn.init.xavier_uniform_(model.linear_relu_stack[4].weight)
    nn.init.zeros_(model.linear_relu_stack[0].bias)
    nn.init.zeros_(model.linear_relu_stack[4].bias)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.321047  [   64/60000]


loss: 1.721905  [ 6464/60000]
loss: 1.693785  [12864/60000]
loss: 1.630021  [19264/60000]
loss: 1.607836  [25664/60000]
loss: 1.625428  [32064/60000]
loss: 1.605342  [38464/60000]
loss: 1.583667  [44864/60000]
loss: 1.556382  [51264/60000]
loss: 1.574652  [57664/60000]
Test Error: 
 Accuracy: 93.4%, Avg loss: 1.544786 

Epoch 2
-------------------------------
loss: 1.491045  [   64/60000]
loss: 1.549093  [ 6464/60000]
loss: 1.534711  [12864/60000]
loss: 1.525746  [19264/60000]
loss: 1.549162  [25664/60000]
loss: 1.526278  [32064/60000]
loss: 1.535664  [38464/60000]
loss: 1.509813  [44864/60000]
loss: 1.518725  [51264/60000]
loss: 1.515718  [57664/60000]
Test Error: 
 Accuracy: 94.8%, Avg loss: 1.522718 

Epoch 3
-------------------------------
loss: 1.514114  [   64/60000]
loss: 1.503289  [ 6464/60000]
loss: 1.540902  [12864/60000]
loss: 1.572305  [19264/60000]
loss: 1.486817  [25664/60000]
loss: 1.523991  [32064/60000]
loss: 1.500935  [38464/60000]
loss: 1.521643  [44864/60000]
loss: 

In [26]:
model.eval()
x, y = test_data[42][0], test_data[42][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = torch.argmax(pred[0]), y
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "4", Actual: "4"
